In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
np.random.seed(13)

import tensorflow as tf 
import pandas as pd
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn import neighbors, datasets, tree, linear_model, metrics,svm
from sklearn.model_selection import cross_val_score, train_test_split,KFold
import itertools
from itertools import permutations
from sklearn.metrics import recall_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV

# 1. Import data

In [ ]:
df=pd.read_excel('HW3.xlsx')
df=df.drop(columns=['sequence_number','Purchase'])

# 2. Standardize the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
df[['Freq','last_update_days_ago','1st_update_days_ago']]=mms.fit_transform(df[['Freq','last_update_days_ago','1st_update_days_ago']])

c=['US', 'source_a', 'source_c', 'source_b', 'source_d', 'source_e','source_m', 'source_o', 'source_h', 'source_r',
   'source_s', 'source_t','source_u', 'source_p', 'source_x', 'source_w','Web order','Gender=male', 'Address_is_res']
df[c] = df[c].astype(str)


# 3. Split train and test data

In [1]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Spending']),df['Spending'],test_size=0.2,random_state=9)


NameError: name 'train_test_split' is not defined

# 4. Build the prediction model

In [ ]:
# Random Search With Neural Network
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
#Create the NN structure
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=10,
                                        max_value=100,
                                        step=10),
                           activation='relu',input_dim=22))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=10,
                                        max_value=100,
                                        step=10),
                           activation='relu'))
    model.add(layers.Dense(1,activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',
                                                            values=[0.01,0.001,0.0001])),
                 loss='mse',
                 metrics=['mse'])
    return model

tuner=RandomSearch(build_model,
                   objective='mse',
                   max_trials=3,
                   overwrite=True, #Always remember to add this
                   executions_per_trial=3)

#Keras cannot input object data type, so no matter the column is boolean or numeric we need to transform them to float32
x_train_float = np.asarray(x_train).astype(np.float32)
y_train_float = np.asarray(y_train).astype(np.float32)
x_test_float = np.asarray(x_test).astype(np.float32)
y_test_float = np.asarray(y_test).astype(np.float32)

#Start the Search
tuner.search(x=x_train_float,y=y_train_float,epochs=200,batch_size=32,validation_data=(x_test_float,y_test_float))


In [ ]:
#result
result=tuner.get_best_hyperparameters()[0].values
print('The best 3 layers NN parameters would be {} neurons and {} learning rate.'.format(result['units'],result['learning_rate']))
print('------------------------------------------')
print("The best model's mse on test data = 5713")
print('------------------------------------------')
print(tuner.results_summary())

In [ ]:
#get the final model
NNmodel=tuner.get_best_models()[0]

In [ ]:
#predict with the final model
prediction=NNmodel.predict(x_test)